In [2]:
%load_ext autoreload
%autoreload 2

In [39]:
%%time
from nngp import NN
from dataloader.rosen import RosenData 

CPU times: user 17 µs, sys: 2 µs, total: 19 µs
Wall time: 22.9 µs


In [36]:
config = {
    'random_state': 4623457,
    'ndim': 10,
    'N_train': 200,
    'N_test': 200,
    'N_pool': 1000,
    'layers': [128, 64, 32]
}



In [37]:
np.random.seed(config['random_state'])

In [40]:
X_train, y_train, _, _, X_test, y_test, X_pool, y_pool = RosenData().dataset(
    config['N_train'], 0, config['N_test'], config['N_pool'], config['ndim']
)

print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

shapes: (200, 10) (200, 1)
shapes: (200, 10) (200, 1)
shapes: (1000, 10) (1000, 1)


In [22]:
tf.reset_default_graph()
nn = NN(
    ndim = ndim,
    random_state = random_state,
    layers = layers
)

In [23]:
try:
    sess.close()
except:
    pass
# a setting for my cluster; ignore it
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# global init
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session(config=config)
sess.run(init)

In [24]:
nn.train(
    session = sess,
    X_train = X_train,
    y_train = y_train,
    X_test = X_test,
    y_test = y_test,
    X_val = X_test,
    y_val = y_test,
)

rmses = [np.sqrt(mse(nn.predict(sess, data = X_test), y_test))]

for al_iteration in range(1, 10):
    print(f'[{al_iteration}] BEFORE:')
    print('shapes:', X_train.shape, y_train.shape)
    print('shapes:', X_test.shape, y_test.shape)
    print('shapes:', X_pool.shape, y_pool.shape)
    
    gpue = nn.get_nngp_UE(sess, X_train, y_train, X_pool, y_pool)
    
    X_train, y_train, X_pool, y_pool = \
        update_learning_sets(X_train,
                             y_train,
                             X_pool,
                             y_pool,
                             gpue,
                             sample_size = 100)
    
    print(f'[{al_iteration}] AFTER:')
    print('shapes:', X_train.shape, y_train.shape)
    print('shapes:', X_test.shape, y_test.shape)
    print('shapes:', X_pool.shape, y_pool.shape)
    
    nn.train(
        session = sess,
        X_train = X_train,
        y_train = y_train,
        X_test = X_test,
        y_test = y_test,
        X_val = X_test,
        y_val = y_test,
    )
    rmses.append(np.sqrt(mse(nn.predict(sess, data = X_test),
            y_test)))

[100] RMSE train:184.986 test:190.024 val:190.024 patience:3
[200] RMSE train:180.968 test:185.993 val:185.993 patience:3
[300] RMSE train:168.460 test:173.408 val:173.408 patience:3
[400] RMSE train:141.652 test:146.424 val:146.424 patience:3
[500] RMSE train:102.373 test:106.928 val:106.928 patience:3
[600] RMSE train:72.031 test:76.324 val:76.324 patience:3
[700] RMSE train:64.877 test:68.781 val:68.781 patience:3
[800] RMSE train:64.360 test:68.179 val:68.179 patience:3
[900] RMSE train:64.193 test:68.075 val:68.075 patience:3
[1000] RMSE train:64.025 test:67.998 val:67.998 patience:3
[1100] RMSE train:63.850 test:67.922 val:67.922 patience:3
[1200] RMSE train:63.669 test:67.845 val:67.845 patience:3
[1300] RMSE train:63.482 test:67.768 val:67.768 patience:3
[1400] RMSE train:63.290 test:67.692 val:67.692 patience:3
[1500] RMSE train:63.094 test:67.616 val:67.616 patience:3
[1600] RMSE train:62.895 test:67.542 val:67.542 patience:3
[1700] RMSE train:62.693 test:67.470 val:67.470 pa

[300] RMSE train:10.761 test:22.336 val:22.336 patience:3
[400] RMSE train:10.228 test:22.219 val:22.219 patience:3
[500] RMSE train:9.758 test:22.145 val:22.145 patience:3
[600] RMSE train:9.337 test:22.072 val:22.072 patience:3
[700] RMSE train:8.947 test:21.968 val:21.968 patience:3
[800] RMSE train:8.561 test:21.852 val:21.852 patience:3
[900] RMSE train:8.204 test:21.755 val:21.755 patience:3
[1000] RMSE train:7.896 test:21.656 val:21.656 patience:3
[1100] RMSE train:7.631 test:21.590 val:21.590 patience:3
[1200] RMSE train:7.383 test:21.514 val:21.514 patience:3
[1300] RMSE train:7.152 test:21.430 val:21.430 patience:3
[1400] RMSE train:6.933 test:21.364 val:21.364 patience:3
[1500] RMSE train:6.720 test:21.305 val:21.305 patience:3
[1600] RMSE train:6.512 test:21.253 val:21.253 patience:3
[1700] RMSE train:6.311 test:21.201 val:21.201 patience:3
[1800] RMSE train:6.118 test:21.161 val:21.161 patience:3
[1900] RMSE train:5.940 test:21.123 val:21.123 patience:3
[2000] RMSE train:5

[1900] RMSE train:7.477 test:17.744 val:17.744 patience:3
[2000] RMSE train:7.426 test:17.712 val:17.712 patience:3
[2100] RMSE train:7.375 test:17.684 val:17.684 patience:3
[2200] RMSE train:7.329 test:17.692 val:17.692 patience:2
[2300] RMSE train:7.275 test:17.645 val:17.645 patience:3
[2400] RMSE train:7.231 test:17.627 val:17.627 patience:3
[2500] RMSE train:7.186 test:17.571 val:17.571 patience:3
[2600] RMSE train:7.142 test:17.553 val:17.553 patience:3
[2700] RMSE train:7.110 test:17.563 val:17.563 patience:2
[2800] RMSE train:7.059 test:17.522 val:17.522 patience:3
[2900] RMSE train:7.025 test:17.453 val:17.453 patience:3
[3000] RMSE train:6.981 test:17.456 val:17.456 patience:2
[3100] RMSE train:6.944 test:17.435 val:17.435 patience:3
[3200] RMSE train:6.926 test:17.456 val:17.456 patience:2
[3300] RMSE train:6.879 test:17.359 val:17.359 patience:3
[3400] RMSE train:6.823 test:17.400 val:17.400 patience:2
[3500] RMSE train:6.782 test:17.352 val:17.352 patience:3
[3600] RMSE tr

In [25]:
rmses

[67.01624050792357,
 25.617914060579913,
 20.79083550341267,
 20.592644604381892,
 19.891202936027778,
 19.169725896655404,
 19.088115981367878,
 18.436352037896626,
 15.836635818902577,
 15.385584932315856]

In [26]:
nn.predict(sess,
           data = X_test[:3])

array([[202.12682],
       [175.21193],
       [360.3318 ]], dtype=float32)

In [27]:
y_test[:3]

array([[183.97308059],
       [186.273626  ],
       [310.47204144]])

In [28]:
gpue = nn.get_nngp_UE(sess, X_train, y_train, X_pool, y_pool)
mcd_ue = nn.get_mcd_UE(sess, X_pool)
gpue, mcd_ue

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([280.89924695, 240.32911521, 212.78439218, 336.67438452,
        371.95826913, 221.99342952, 239.36145661, 175.08057346,
        250.84419167, 334.66508059, 183.36869235, 205.28629383,
        202.90857136, 292.08283795, 174.96503579, 360.71784888,
        194.59085206, 110.29008744, 270.02842567, 272.92573225,
        283.68638707, 228.97726692, 169.21232216, 241.86923846,
        268.76122167, 182.70312032, 208.02437352, 217.26213109,
        285.40837133, 244.2959037 , 254.17450913, 227.31588614,
        324.9356297 , 315.27853122, 24

In [29]:
print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

shapes: (1100, 10) (1100, 1)
shapes: (200, 10) (200, 1)
shapes: (100, 10) (100, 1)


In [30]:
X_train, y_train, X_pool, y_pool = \
    update_learning_sets(X_train,
                         y_train,
                         X_pool,
                         y_pool,
                         gpue,
                         sample_size = 10)

In [31]:
print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

shapes: (1110, 10) (1110, 1)
shapes: (200, 10) (200, 1)
shapes: (90, 10) (90, 1)
